### N Gram In N Gram Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding native word to shared word
twogram_start = 0  # 0  # native word start index
twogram_end = 1000  # 28  # native word end index

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False

# n gram sample
threegram_sample = 6
fourgram_sample = 4
fivegram_sample = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

In [4]:
#path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Deployment2/Result/5-N Gram In N Gram Analysis/{lang_folder.capitalize()} {lang_pair.capitalize()}"
#
#Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_in_wordgroup(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

In [7]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
    df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]
    df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]

df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
995,ha siktir,10746
996,gidelim mi,10744
997,olarak ne,10726
998,bunu yapamazsın,10724


In [8]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [9]:
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram = df_fourgram_sent.loc[:,["fourgram","frequency"]]
else:
    df_fourgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")  
    df_fourgram = df_fourgram.loc[:,["fourgram","frequency"]]

df_fourgram

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [10]:
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram = df_fivegram_sent.loc[:,["fivegram","frequency"]]
else:
    df_fivegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")  
    df_fivegram = df_fivegram.loc[:,["fivegram","frequency"]]

df_fivegram

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [11]:
df_ngram_concat = pd.concat([df_twogram_select, df_threegram, df_fourgram, df_fivegram], axis=1)
df_ngram_concat

,twogram,frequency,threegram,frequency,fourgram,frequency,fivegram,frequency
0,bir şey,859944.0,bir şey yok,113165.0,bir şey var mı,41773,başka bir şey var mı,14104.0
1,değil mi,585879.0,bir şey var,110455.0,her şey yolunda mı,31126,bu da ne demek oluyor,10205.0
2,ben de,377765.0,bu da ne,89463.0,burada ne işin var,21993,o kadar da kötü değil,7012.0
3,teşekkür ederim,370619.0,her şey yolunda,75968.0,bir sorun mu var,21423,sence de öyle değil mi,6305.0
4,ne oldu,322758.0,başka bir şey,75193.0,ben de seni seviyorum,17338,sana bir şey sorabilir miyim,6224.0
...,...,...,...,...,...,...,...,...
3052209,NaN,NaN,NaN,NaN,fikret miloya merhaba de,5,NaN,NaN
3052210,NaN,NaN,NaN,NaN,fikret millsin oğlu mu,5,NaN,NaN
3052211,NaN,NaN,NaN,NaN,fikret millet iyi iş,5,NaN,NaN
3052212,NaN,NaN,NaN,NaN,fikret millet bize bakıyor,5,NaN,NaN


In [12]:
df_two_in_three = word_in_wordgroup(df_ngram_concat, "twogram","threegram",threegram_sample)
df_two_in_four = word_in_wordgroup(df_ngram_concat, "twogram","fourgram",fourgram_sample)
df_two_in_five = word_in_wordgroup(df_ngram_concat, "twogram","fivegram",fivegram_sample)

In [13]:
df_twogram_order_join_threegram = df_two_in_three.groupby(["twogram"])["threegram"].apply(", ".join).reset_index()
df_twogram_order_join_fourgram = df_two_in_four.groupby(["twogram"])["fourgram"].apply(", ".join).reset_index()
df_twogram_order_join_fivegram = df_two_in_five.groupby(["twogram"])["fivegram"].apply(", ".join).reset_index()  

In [14]:
dfs = [df_twogram_order_join_threegram, df_twogram_order_join_fourgram, df_twogram_order_join_fivegram]

In [15]:
df_ngram_sample_join_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='outer'), dfs)
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge 

,twogram,threegram,fourgram,fivegram
0,acele edin,"hadi acele edin, lütfen acele edin, haydi acel...","acele edin acele edin, hadi çocuklar acele edi...","acele edin 15 dakika sonra, acele edin ve geti..."
1,acele et,"hadi acele et, acele et ve, haydi acele et, lü...","acele et acele et, acele et o zaman, o zaman a...","acele et ve buraya gel, acele et nong fikret f..."
2,adil değil,"hiç adil değil, bu adil değil, de adil değil, ...","bu hiç adil değil, hiç de adil değil, ama bu a...","bu hiç de adil değil, ama bu hiç adil değil"
3,adı ne,"onun adı ne, adı ne demiştin, adı ne peki, bun...","adı ne olursa olsun, eden beyefendinin adı ne,...","eşlik eden beyefendinin adı ne, bunun adı ne o..."
4,adım fikret,"benim adım fikret, adım fikret fikret, adım fi...","benim adım fikret fikret, adım fikret ve ben, ...","adım fikret ve ben bir, benim adım fikret fikr..."
...,...,...,...,...
995,şu ki,"gerçek şu ki, sorun şu ki, mesele şu ki, olay ...","ama gerçek şu ki, ama sorun şu ki, i̇yi haber ...","ama gerçek şu ki o, ama iyi haber şu ki"
996,şuna bak,"hey şuna bak, tanrım şuna bak, de şuna bak, şu...","bir de şuna bak, şuna bak şuna bak, gel de şun...","şuna bak ne kadar da, hey fikret şuna bak bakalım"
997,şuna bakın,"hey şuna bakın, şuna bakın hele, çocuklar şuna...","bir de şuna bakın, vay canına şuna bakın, hey ...","şuna bakın ne kadar da, şuna bakın görmeyeli n..."
998,şuna bir,"şuna bir bak, şuna bir bakın, şuna bir baksana...","şuna bir göz at, şuna bir bakar mısın, gel de ...","gel de şuna bir bak, şuna bir baksan iyi olur"


In [16]:
df_ngram_sample_join_merge = pd.merge(df_ngram_sample_join_merge, df_twogram_select, how="left", on="twogram")
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge.rename(columns={"frequency":"two_freq"}, inplace=True)
df_ngram_sample_join_merge.sort_values(by="two_freq", ascending=False, inplace=True)
df_ngram_sample_join_merge.reset_index(drop=True, inplace=True)
df_ngram_sample_join_merge

,twogram,threegram,fourgram,fivegram,two_freq
0,bir şey,"bir şey yok, bir şey var, başka bir şey, bir ş...","bir şey var mı, gereken bir şey var, başka bir...","başka bir şey var mı, sana bir şey sorabilir m...",859944
1,değil mi,"öyle değil mi, biliyorsun değil mi, güzel deği...","de öyle değil mi, bir şey değil mi, çok güzel ...","sence de öyle değil mi, sizce de öyle değil mi",585879
2,ben de,"ben de seni, ben de öyle, ve ben de, evet ben ...","ben de seni seviyorum, ben de öyle düşünmüştüm...","ben de seni gördüğüme sevindim, evet ben de öy...",377765
3,teşekkür ederim,"çok teşekkür ederim, için teşekkür ederim, teş...","için çok teşekkür ederim, geldiğiniz için teşe...","her şey için teşekkür ederim, zaman ayırdığını...",370619
4,ne oldu,"sonra ne oldu, ne oldu sana, sana ne oldu, ne ...","bil bakalım ne oldu, sana ne oldu böyle, ne ol...","az önce ne oldu öyle, sonra ne oldu biliyor musun",322758
...,...,...,...,...,...
995,ha siktir,"ha siktir ya, ha siktir be, ha siktir oradan, ...","ha siktir zavala sen, ha siktir oradan bu, ha ...","ha siktir zavala sen yakala, ha siktir oradan ...",10746
996,gidelim mi,"gidelim mi artık, yere gidelim mi, içmeye gide...","bir yere gidelim mi, şeyler içmeye gidelim mi,...","başka bir yere gidelim mi, bir şeyler içmeye g...",10744
997,olarak ne,"tam olarak ne, olarak ne olduğunu, olarak ne k...","tam olarak ne olduğunu, tam olarak ne demek, t...","tam olarak ne demek istiyorsun, bu tam olarak ...",10726
998,bunu yapamazsın,"hayır bunu yapamazsın, bana bunu yapamazsın, f...","bunu yapamazsın tamam mı, çünkü bunu yapamazsı...","çünkü bunu yapamazsın tamam mı, artık bunu yap...",10724


In [17]:
#df_ngram_sample_join_merge.to_excel(f"{twogram_end}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
#Fourgram_{fivegram_sample}_Fivegram_Sample_Join_Result.xlsx", index=False)

#### Adding Frequency For Ratio

In [18]:
df_two_in_three_freq = pd.merge(df_two_in_three, df_threegram, how="left", on="threegram")
df_two_in_three_freq.drop_duplicates(inplace=True)
df_two_in_three_freq.rename(columns={"frequency":"three_freq"}, inplace=True)
df_two_in_three_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_three_freq

,threegram,three_freq
0,bir şey yok,113165
1,bir şey var,110455
2,başka bir şey,75193
3,bir şey değil,68498
4,bir şey mi,64933
...,...,...
5994,sorun olmaz mı,823
5995,konuşsak olmaz mı,323
5996,güzel olmaz mı,230
5997,konuşmasak olmaz mı,218


In [19]:
df_two_in_four_freq = pd.merge(df_two_in_four, df_fourgram, how="left", on="fourgram")
df_two_in_four_freq.drop_duplicates(inplace=True)
df_two_in_four_freq.rename(columns={"frequency":"four_freq"}, inplace=True)
df_two_in_four_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_four_freq

,fourgram,four_freq
0,bir şey var mı,41773
1,gereken bir şey var,15683
2,başka bir şey var,12619
3,bir şey daha var,12185
4,de öyle değil mi,5379
...,...,...
3991,hadi ama bunu yapamazsın,19
3992,daha iyi olmaz mı,1415
3993,daha kolay olmaz mı,176
3994,için sorun olmaz mı,121


In [20]:
df_two_in_five_freq = pd.merge(df_two_in_five, df_fivegram, how="left", on="fivegram")
df_two_in_five_freq.drop_duplicates(inplace=True)
df_two_in_five_freq.rename(columns={"frequency":"five_freq"}, inplace=True)
df_two_in_five_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_five_freq

,fivegram,five_freq
0,başka bir şey var mı,14104
1,sana bir şey sorabilir miyim,6224
2,sence de öyle değil mi,6305
3,sizce de öyle değil mi,1930
4,ben de seni gördüğüme sevindim,1641
...,...,...
1991,bu tam olarak ne demek,284
1992,çünkü bunu yapamazsın tamam mı,25
1993,artık bunu yapamazsın tamam mı,18
1994,senin için sorun olmaz mı,111


In [21]:
df_ngram_sample_concat = pd.concat([df_twogram_select,df_two_in_three_freq, df_two_in_four_freq, df_two_in_five_freq], axis=1)
df_ngram_sample_concat

,twogram,frequency,threegram,three_freq,fourgram,four_freq,fivegram,five_freq
0,bir şey,859944.0,bir şey yok,113165,bir şey var mı,41773.0,başka bir şey var mı,14104.0
1,değil mi,585879.0,bir şey var,110455,gereken bir şey var,15683.0,sana bir şey sorabilir miyim,6224.0
2,ben de,377765.0,başka bir şey,75193,başka bir şey var,12619.0,sence de öyle değil mi,6305.0
3,teşekkür ederim,370619.0,bir şey değil,68498,bir şey daha var,12185.0,sizce de öyle değil mi,1930.0
4,ne oldu,322758.0,bir şey mi,64933,de öyle değil mi,5379.0,ben de seni gördüğüme sevindim,1641.0
...,...,...,...,...,...,...,...,...
5994,NaN,NaN,sorun olmaz mı,823,NaN,NaN,NaN,NaN
5995,NaN,NaN,konuşsak olmaz mı,323,NaN,NaN,NaN,NaN
5996,NaN,NaN,güzel olmaz mı,230,NaN,NaN,NaN,NaN
5997,NaN,NaN,konuşmasak olmaz mı,218,NaN,NaN,NaN,NaN


In [22]:
#df_ngram_sample_concat.to_excel(f"{twogram_end}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
#Fourgram_{fivegram_sample}_Fivegram_Sample_With_Frequency_Result.xlsx", index=False)

#### Calculate Result And Select Ratio

In [23]:
twogram_result_freq = df_twogram_select["frequency"].sum()
threegram_result_freq = df_ngram_sample_concat["three_freq"].sum()
fourgram_result_freq = df_ngram_sample_concat["four_freq"].sum()
fivegram_result_freq = df_ngram_sample_concat["five_freq"].sum()

In [24]:
twogram_select_freq = df_twogram.iloc[twogram_select_start:twogram_select_end,]["frequency"].sum()
threegram_select_freq = df_threegram.iloc[threegram_select_start:threegram_select_end,]["frequency"].sum()
fourgram_select_freq = df_fourgram.iloc[fourgram_select_start:fourgram_select_end,]["frequency"].sum()
fivegram_select_freq = df_fivegram.iloc[fivegram_select_start:fivegram_select_end,]["frequency"].sum()

In [25]:
(twogram_result_freq/twogram_select_freq)*100

53.772282587898346

In [26]:
(threegram_result_freq/threegram_select_freq)*100

81.65794790486302

In [27]:
(fourgram_result_freq/fourgram_select_freq)*100

72.590010787832

In [28]:
(fivegram_result_freq/fivegram_select_freq)*100

50.810291930960496

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{twogram_end}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_*_Result.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass